# Загрузить датасет

In [1]:
# Добавляем импорты

from gensim.models import FastText
import pandas as pd
import numpy as np
import pymorphy2
import nltk  # Natural Language Toolkit - для загрузки стоп слов русского языка
import re

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm  # прогресбарbincount

from sklearn.model_selection import train_test_split  # рабиваем данные на traint и test

### Создаем утилиты для работы с датасетом

In [2]:
regex = re.compile(r'[А-Яа-яA-zёЁ-]+')
def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text)).lower()
    except Exception as e:
        return ""

In [65]:
def process_data(data, all_stop_words) -> []:
    word_tokenizer = nltk.WordPunctTokenizer()

    texts = []
    targets = []

    # инициализируем лемматизатор
    morph = pymorphy2.MorphAnalyzer()

    # поочередно проходим по всем новостям в списке
    for item in tqdm(data):
        text_lower = words_only(item)  # оставим только слова
        tokens = word_tokenizer.tokenize(text_lower)  # разбиваем текст на слова

        # удаляем пунктуацию и стоп-слова, а так же лемитизируем  слова
        tokens = [morph.parse(word)[0].normal_form for word in tokens if
                  (word not in all_stop_words and not word.isnumeric())]

        # texts.append(tokens) # добавляем в предобработанный список
        texts.append(' '.join(tokens).strip())

    return texts

In [5]:
def read_prepared_datasets(file_name) -> []:
    result = []
    with open(file_name, 'r', encoding='utf-8') as file:
        for line in file.readlines():
            # result.append(line.replace('\n', ''))
            result.append(line)
    return result

In [6]:
# загружаем список стоп-слов для русского
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/avetall87/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
prepared_dataset_file_name = 'dataset/prepared_dataset.txt'

# расширим список стоп-слов, словами, которые являеются стоп-словами в данной задаче
add_stop_words = ['ао', 'оао', 'ооо']
months = ['январь', 'февраль', 'март', 'апрель', 'май', 'июнь', 'июль', 'август', 'сентябрь', 'октябрь', 'ноябрь',
            'декабрь', ]
all_stop_words = stop_words + add_stop_words + months

df = pd.read_csv('dataset/dataset.csv', sep=',', engine='python')

y = df['title'].tolist()

In [8]:
texts = read_prepared_datasets(prepared_dataset_file_name)
len(texts)
texts[0:3]

['подвижный сбор год днёвка деревня катеринный\n',
 'герой советский союз лётчик медноног скульптор м аникушин генеральный консул народный республика болгария ленинград вангел петрович трынок слева направо мастерская скульптор\n',
 'дважды герой соц труд смирнов избирательный участок время выборы местный совет народный судья\n']

# Создаем модель FastText и обучаем ее

In [12]:
prepared_vocabulary = []

for words in texts:
  splitted_data = words.split(' ')
  prepared_vocabulary.append(splitted_data)

In [13]:
model = FastText(prepared_vocabulary,min_count=1, vector_size=100, workers=4)

In [16]:
model.save('./model/fasttext_pretrained.bin')

In [17]:
#TF-IDF # needs a list of lists for words and docs along with a fasttext 'model'
text = []
for i in tqdm(prepared_vocabulary):
  string = ' '.join(i)
  text.append(string)
tf_idf_vect = TfidfVectorizer(stop_words=None)
final_tf_idf = tf_idf_vect.fit_transform(text)
tfidf_feat = list(tf_idf_vect.get_feature_names_out())

100%|██████████| 116000/116000 [00:00<00:00, 925554.98it/s]


In [18]:
tfidf_sent_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list

row=0;
errors=0
for sent in tqdm(texts): # for each review/sentence
    sent_vec = np.zeros(100) # as word vectors are of zero length
    try:
        weight_sum =0; # num of words with a valid vector in the sentence/review
        for word in sent.split(' '): # for each word in a review/sentence
            word = word.replace('\n', '') # костыль
            try:
                vec = model.wv[word]
                # obtain the tf_idfidf of a word in a sentence/review
                tfidf = final_tf_idf [row, tfidf_feat.index(word)]
                sent_vec += (vec * tfidf)
                weight_sum += tfidf
            except Exception as e:
                errors =+1
        sent_vec /= weight_sum
        tfidf_sent_vectors.append(sent_vec)
        row += 1
    except Exception as e:
        errors =+1
        row += 1
print('errors noted: '+str(errors))

 82%|████████▏ | 95232/116000 [03:43<00:47, 435.24it/s]/tmp/ipykernel_781707/3216452434.py:19: RuntimeWarning: invalid value encountered in divide
  sent_vec /= weight_sum
100%|██████████| 116000/116000 [04:33<00:00, 424.33it/s]

errors noted: 1


In [19]:
df['FT_tfidf'] = tfidf_sent_vectors

In [20]:
df.head(10)

,title,text,FT_tfidf
0,6_KffdPhotoDoc_65243,Подвижной сбор 1901 года.Дневка у деревни Кате...,"[0.4116544160848603, 1.0964632404917214, -0.35..."
1,6_KffdPhotoDoc_941,"Герой Советского Союза, летчик В.А.Медноногов,...","[-0.11494636164310777, -0.0835563225234515, 0...."
2,6_KffdPhotoDoc_6002,Дважды Герой Соц.Труда В.А.Смирнов на избирате...,"[-0.37379294465812923, -0.18142475095465768, 0..."
3,6_KffdPhotoDoc_65096,Вид сверху на казармы полка.,"[-0.27052257421680426, 2.7671281437643973, 0.8..."
4,6_KffdPhotoDoc_95843,Генерал казачьих войск Холмский (сидит 2-ой сл...,"[-0.7048170104829009, 1.1691046676297512, 1.43..."
5,6_KffdPhotoDoc_66745,Секции армокаркаса у подножия бокового корпуса.,"[-0.19617305594635462, 1.14377264431047, 0.266..."
6,6_KffdPhotoDoc_58622,Кавалерийские части на параде войск проходят м...,"[-0.6553515271295852, 0.6134593769388038, 1.54..."
7,6_KffdPhotoDoc_71835,Пассажирский поезд прибыл на Финляндский вокзал.,"[-0.7451638977312363, 0.19082692085069672, 1.2..."
8,6_KffdPhotoDoc_112688,"Хирург, профессор, орденоносец И.П.Виноградов ...","[-0.6193937706347936, 0.3283097044444246, 0.87..."
9,6_KffdPhotoDoc_18624,Скульптор В.Б. Пинчук за работой над скульптур...,"[-0.01576843721590673, 0.5509282297272745, 0.6..."


In [21]:
# Сохраняем данные в файл
df.to_csv(path_or_buf='dataset/result.csv', encoding='utf-8', index_label=['title', 'FT_tfidf'])

In [15]:
df = pd.read_csv('dataset/result.csv', sep=',', engine='python')

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/result.csv'

In [22]:
def get_vector_by_text(text) -> []:
    tfidf_sent_vectors = []

    tokenizer = TfidfVectorizer(stop_words=None)
    tokenized_data = tokenizer.fit_transform(text)
    feature_names = list(tokenizer.get_feature_names_out())

    sent_vec = np.zeros(100) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in text:
        vector = model.wv[word]
        # obtain the tf_idfidf of a word in a sentence/review
        tfidf = tokenized_data [0, feature_names.index(word)]
        sent_vec += (vector * tfidf)
        weight_sum += tfidf
        sent_vec /= weight_sum
    tfidf_sent_vectors.append(sent_vec)
    return tfidf_sent_vectors
    


In [88]:
from sklearn.metrics.pairwise import cosine_similarity

search_query = 'скульптор работой'.lower().split(' ')

query = get_vector_by_text(process_data(search_query, all_stop_words))

query = np.array(list(query))
query = np.nan_to_num(query)
print(query)

vectors = np.array(list(df.FT_tfidf.values))
vectors = np.nan_to_num(vectors)
print(vectors)

cosine_similarities = pd.Series(cosine_similarity(query, vectors).flatten())

for i,j in cosine_similarities.nlargest(3).items():
  print({'weight':str(j), 'entity_chiper':df.title.iloc[i], 'raw text':df.text.iloc[i]})

100%|██████████| 2/2 [00:00<00:00, 3232.60it/s]


[[ 7.64364719e-01  9.18429852e-01  1.14995551e+00  7.27227271e-01
  -2.52510160e-01  1.87241006e+00 -4.20541674e-01  5.78596473e-01
  -3.89921576e-01 -8.59337211e-01  1.38342083e+00  1.87447441e+00
  -1.08938241e+00  6.39604867e-01  2.02597976e+00  1.54741168e+00
  -1.06618571e+00  2.96719909e+00 -2.49364114e+00 -1.76956928e+00
   8.24022472e-01  2.32724738e+00  2.63922572e+00  8.27197492e-01
  -9.92500842e-01  1.24411416e+00  7.31178746e-02 -1.82874095e+00
   8.85768771e-01  2.57537317e+00 -4.47489470e-01 -1.06339562e+00
  -1.22363114e+00  2.28562683e-01 -1.97889173e+00 -1.43619335e+00
  -3.20266175e+00  1.30090451e+00  2.98336744e+00  1.53075075e+00
  -9.77540910e-01 -8.59601736e-01 -1.16594207e+00 -3.30367982e-01
  -8.19887400e-01  1.09220588e+00 -1.76278079e+00  1.83768344e+00
  -4.94398475e-01 -1.82354307e+00  1.05427682e+00  1.06460643e+00
  -4.44970606e-03  3.72769535e-01  3.10886550e+00  2.73102641e+00
  -2.09745550e+00 -1.11150563e+00  6.92720175e-01  1.43241575e-02
   5.62252